# Adversarial Networks Survey - Training

The models output are set to 1000 classes, because we are using the CIFAR 10 dataset we will need to train the network for this dataset

## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import copy
import time
import os
from tqdm import tqdm

In [2]:
datasetModels = ["alexnet", "googlenet", "mobilenet"]

datasetAlgorithm = ["deepfool", "fgsm", "jsma"]

datasetAppend = ["_train.pt", "_test.pt"]

stateDictNames = ["alexnet_state_dict", "googlenet_state_dict", "mobilenet_state_dict"]

In [3]:
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    """Custom dataset."""

    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        sample = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)

        return sample, label

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((70,70)),
    transforms.RandomCrop((64,64)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

transformUnnormalize = transforms.Compose({
    transforms.Normalize((0, 0, 0), (1/0.229, 1/0.224, 1/0.225)),
    transforms.Normalize((-0.485, -0.456, -0.406), (1, 1, 1))
})

batch_size = 1000

criterion = nn.CrossEntropyLoss()

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [5]:
if not os.path.isfile("models/models.pt"):
    raise FileNotFoundError("File 'models.pt' not found in folder models. Please run initialTraining.ipynb to generate the file needed! (File not included due to size)")

trained_weights_cifar10 = torch.load("models/models.pt")

modelNames = ["alexnet", "googlenet", "mobilenet"]

In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, device="cpu"):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

## Alexnet Training

In [7]:
model = torchvision.models.alexnet(weights=torchvision.models.AlexNet_Weights.IMAGENET1K_V1)
num_feats = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_feats, 10)
nameIdx = 0

### DeepFool

In [8]:
model.load_state_dict(trained_weights_cifar10[stateDictNames[nameIdx]])
algorithmIdx = 0

In [9]:
checkpoint = torch.load("perturbedDataset/"+datasetModels[nameIdx]+"_"+datasetAlgorithm[algorithmIdx]+datasetAppend[0])
perturbedImgCollection = checkpoint["perturbedImgCollection"]
perturbedImgCollection.requires_grad = False
perturbedLabelCollection = checkpoint["perturbedLabelCollection"]
gtLabelCollection = checkpoint["gtLabelCollection"]
gtLabelCollection = gtLabelCollection.long()

trainset = CustomDataset(perturbedImgCollection, gtLabelCollection, transform=None)
trainset, valset = torch.utils.data.random_split(trainset, [8000, 2000])

trainload = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)
valload = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=1)

dataloaders = {"train": trainload, "val": valload}
dataset_sizes = {"train": len(trainset), "val": len(valset)}

In [10]:
model.to(device)
model.train()

optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [11]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=25, device=device)
# alexnet = train_model(alexnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.9043 Acc: 0.5092
val Loss: 0.7756 Acc: 0.6415

Epoch 1/24
----------
train Loss: 0.7694 Acc: 0.6440
val Loss: 0.6898 Acc: 0.7070

Epoch 2/24
----------
train Loss: 0.6460 Acc: 0.7284
val Loss: 0.6196 Acc: 0.7645

Epoch 3/24
----------
train Loss: 0.5686 Acc: 0.7780
val Loss: 0.5687 Acc: 0.7865

Epoch 4/24
----------
train Loss: 0.5111 Acc: 0.8041
val Loss: 0.5354 Acc: 0.8025

Epoch 5/24
----------
train Loss: 0.4708 Acc: 0.8249
val Loss: 0.5156 Acc: 0.8125

Epoch 6/24
----------
train Loss: 0.4276 Acc: 0.8425
val Loss: 0.5017 Acc: 0.8195

Epoch 7/24
----------
train Loss: 0.4049 Acc: 0.8556
val Loss: 0.5006 Acc: 0.8200

Epoch 8/24
----------
train Loss: 0.4036 Acc: 0.8585
val Loss: 0.4994 Acc: 0.8195

Epoch 9/24
----------
train Loss: 0.4052 Acc: 0.8549
val Loss: 0.4984 Acc: 0.8200

Epoch 10/24
----------
train Loss: 0.4014 Acc: 0.8560
val Loss: 0.4973 Acc: 0.8195

Epoch 11/24
----------
train Loss: 0.3972 Acc: 0.8546
val Loss: 0.4962 Acc: 0.8200

Ep

In [12]:
torch.save(model.state_dict(),"models/"+datasetModels[nameIdx]+"_"+datasetAlgorithm[algorithmIdx]+".pt")

### FGSM

In [13]:
model.load_state_dict(trained_weights_cifar10[stateDictNames[nameIdx]])
algorithmIdx = 1

In [14]:
checkpoint = torch.load("perturbedDataset/"+datasetModels[nameIdx]+"_"+datasetAlgorithm[algorithmIdx]+datasetAppend[0])
perturbedImgCollection = checkpoint["perturbedImgCollection"]
perturbedImgCollection.requires_grad = False
perturbedLabelCollection = checkpoint["perturbedLabelCollection"]
gtLabelCollection = checkpoint["gtLabelCollection"]
gtLabelCollection = gtLabelCollection.long()

trainset = CustomDataset(perturbedImgCollection, gtLabelCollection, transform=None)
trainset, valset = torch.utils.data.random_split(trainset, [8000, 2000])

trainload = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)
valload = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=1)

dataloaders = {"train": trainload, "val": valload}
dataset_sizes = {"train": len(trainset), "val": len(valset)}

In [15]:
model.to(device)
model.train()

optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [16]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=25, device=device)
# alexnet = train_model(alexnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 1.2727 Acc: 0.6591
val Loss: 1.0782 Acc: 0.6930

Epoch 1/24
----------
train Loss: 1.2132 Acc: 0.6655
val Loss: 1.0268 Acc: 0.6990

Epoch 2/24
----------
train Loss: 1.1010 Acc: 0.6827
val Loss: 0.9784 Acc: 0.7050

Epoch 3/24
----------
train Loss: 1.0140 Acc: 0.6913
val Loss: 0.9360 Acc: 0.7100

Epoch 4/24
----------
train Loss: 0.9446 Acc: 0.7051
val Loss: 0.8998 Acc: 0.7090

Epoch 5/24
----------
train Loss: 0.8785 Acc: 0.7206
val Loss: 0.8741 Acc: 0.7110

Epoch 6/24
----------
train Loss: 0.8414 Acc: 0.7216
val Loss: 0.8533 Acc: 0.7185

Epoch 7/24
----------
train Loss: 0.8049 Acc: 0.7281
val Loss: 0.8517 Acc: 0.7185

Epoch 8/24
----------
train Loss: 0.8003 Acc: 0.7324
val Loss: 0.8502 Acc: 0.7190

Epoch 9/24
----------
train Loss: 0.8035 Acc: 0.7291
val Loss: 0.8488 Acc: 0.7190

Epoch 10/24
----------
train Loss: 0.8026 Acc: 0.7330
val Loss: 0.8474 Acc: 0.7195

Epoch 11/24
----------
train Loss: 0.7923 Acc: 0.7374
val Loss: 0.8461 Acc: 0.7205

Ep

In [17]:
torch.save(model.state_dict(),"models/"+datasetModels[nameIdx]+"_"+datasetAlgorithm[algorithmIdx]+".pt")

### JSMA

In [18]:
model.load_state_dict(trained_weights_cifar10[stateDictNames[nameIdx]])
algorithmIdx = 2

In [19]:
checkpoint = torch.load("perturbedDataset/"+datasetModels[nameIdx]+"_"+datasetAlgorithm[algorithmIdx]+datasetAppend[0])
perturbedImgCollection = checkpoint["perturbedImgCollection"]
perturbedImgCollection.requires_grad = False
perturbedLabelCollection = checkpoint["perturbedLabelCollection"]
gtLabelCollection = checkpoint["gtLabelCollection"]
gtLabelCollection = gtLabelCollection.long()

trainset = CustomDataset(perturbedImgCollection, gtLabelCollection, transform=None)
trainset, valset = torch.utils.data.random_split(trainset, [8000, 2000])

trainload = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)
valload = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=1)

dataloaders = {"train": trainload, "val": valload}
dataset_sizes = {"train": len(trainset), "val": len(valset)}

In [20]:
model.to(device)
model.train()

optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [21]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=25, device=device)
# alexnet = train_model(alexnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.2431 Acc: 0.9319
val Loss: 0.2416 Acc: 0.9295

Epoch 1/24
----------
train Loss: 0.2410 Acc: 0.9351
val Loss: 0.2405 Acc: 0.9300

Epoch 2/24
----------
train Loss: 0.2312 Acc: 0.9356
val Loss: 0.2396 Acc: 0.9305

Epoch 3/24
----------
train Loss: 0.2177 Acc: 0.9365
val Loss: 0.2392 Acc: 0.9305

Epoch 4/24
----------
train Loss: 0.2093 Acc: 0.9400
val Loss: 0.2392 Acc: 0.9310

Epoch 5/24
----------
train Loss: 0.2053 Acc: 0.9393
val Loss: 0.2391 Acc: 0.9310

Epoch 6/24
----------
train Loss: 0.1936 Acc: 0.9454
val Loss: 0.2389 Acc: 0.9315

Epoch 7/24
----------
train Loss: 0.1898 Acc: 0.9435
val Loss: 0.2389 Acc: 0.9310

Epoch 8/24
----------
train Loss: 0.1925 Acc: 0.9427
val Loss: 0.2388 Acc: 0.9315

Epoch 9/24
----------
train Loss: 0.1890 Acc: 0.9434
val Loss: 0.2388 Acc: 0.9310

Epoch 10/24
----------
train Loss: 0.1896 Acc: 0.9460
val Loss: 0.2388 Acc: 0.9305

Epoch 11/24
----------
train Loss: 0.1904 Acc: 0.9451
val Loss: 0.2387 Acc: 0.9310

Ep

In [22]:
torch.save(model.state_dict(),"models/"+datasetModels[nameIdx]+"_"+datasetAlgorithm[algorithmIdx]+".pt")

## GoogLeNet Training

In [23]:
model = torchvision.models.googlenet(weights=torchvision.models.GoogLeNet_Weights.IMAGENET1K_V1)
num_feats = model.fc.in_features
model.fc = nn.Linear(num_feats, 10)
nameIdx = 1

### DeepFool

In [24]:
model.load_state_dict(trained_weights_cifar10[stateDictNames[nameIdx]])
algorithmIdx = 0

In [25]:
checkpoint = torch.load("perturbedDataset/"+datasetModels[nameIdx]+"_"+datasetAlgorithm[algorithmIdx]+datasetAppend[0])
perturbedImgCollection = checkpoint["perturbedImgCollection"]
perturbedImgCollection.requires_grad = False
perturbedLabelCollection = checkpoint["perturbedLabelCollection"]
gtLabelCollection = checkpoint["gtLabelCollection"]
gtLabelCollection = gtLabelCollection.long()

trainset = CustomDataset(perturbedImgCollection, gtLabelCollection, transform=None)
trainset, valset = torch.utils.data.random_split(trainset, [8000, 2000])

trainload = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)
valload = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=1)

dataloaders = {"train": trainload, "val": valload}
dataset_sizes = {"train": len(trainset), "val": len(valset)}

In [26]:
model.to(device)
model.train()

optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [27]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=25, device=device)
# alexnet = train_model(alexnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.6989 Acc: 0.6710
val Loss: 0.6587 Acc: 0.7520

Epoch 1/24
----------
train Loss: 0.5819 Acc: 0.7590
val Loss: 0.5345 Acc: 0.8030

Epoch 2/24
----------
train Loss: 0.4782 Acc: 0.8261
val Loss: 0.4333 Acc: 0.8565

Epoch 3/24
----------
train Loss: 0.3920 Acc: 0.8731
val Loss: 0.3601 Acc: 0.8880

Epoch 4/24
----------
train Loss: 0.3360 Acc: 0.9038
val Loss: 0.3193 Acc: 0.9045

Epoch 5/24
----------
train Loss: 0.3035 Acc: 0.9111
val Loss: 0.2958 Acc: 0.9125

Epoch 6/24
----------
train Loss: 0.2819 Acc: 0.9201
val Loss: 0.2801 Acc: 0.9170

Epoch 7/24
----------
train Loss: 0.2691 Acc: 0.9273
val Loss: 0.2760 Acc: 0.9190

Epoch 8/24
----------
train Loss: 0.2679 Acc: 0.9261
val Loss: 0.2739 Acc: 0.9205

Epoch 9/24
----------
train Loss: 0.2653 Acc: 0.9274
val Loss: 0.2726 Acc: 0.9220

Epoch 10/24
----------
train Loss: 0.2648 Acc: 0.9303
val Loss: 0.2717 Acc: 0.9225

Epoch 11/24
----------
train Loss: 0.2645 Acc: 0.9303
val Loss: 0.2707 Acc: 0.9230

Ep

In [28]:
torch.save(model.state_dict(),"models/"+datasetModels[nameIdx]+"_"+datasetAlgorithm[algorithmIdx]+".pt")

### FGSM

In [29]:
model.load_state_dict(trained_weights_cifar10[stateDictNames[nameIdx]])
algorithmIdx = 1

In [30]:
checkpoint = torch.load("perturbedDataset/"+datasetModels[nameIdx]+"_"+datasetAlgorithm[algorithmIdx]+datasetAppend[0])
perturbedImgCollection = checkpoint["perturbedImgCollection"]
perturbedImgCollection.requires_grad = False
perturbedLabelCollection = checkpoint["perturbedLabelCollection"]
gtLabelCollection = checkpoint["gtLabelCollection"]
gtLabelCollection = gtLabelCollection.long()

trainset = CustomDataset(perturbedImgCollection, gtLabelCollection, transform=None)
trainset, valset = torch.utils.data.random_split(trainset, [8000, 2000])

trainload = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)
valload = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=1)

dataloaders = {"train": trainload, "val": valload}
dataset_sizes = {"train": len(trainset), "val": len(valset)}

In [31]:
model.to(device)
model.train()

optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [32]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=25, device=device)
# alexnet = train_model(alexnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 1.5355 Acc: 0.5773
val Loss: 1.4592 Acc: 0.5745

Epoch 1/24
----------
train Loss: 1.4175 Acc: 0.5968
val Loss: 1.3673 Acc: 0.6065

Epoch 2/24
----------
train Loss: 1.2659 Acc: 0.6304
val Loss: 1.2373 Acc: 0.6370

Epoch 3/24
----------
train Loss: 1.1063 Acc: 0.6558
val Loss: 1.0894 Acc: 0.6710

Epoch 4/24
----------
train Loss: 0.9582 Acc: 0.6960
val Loss: 0.9508 Acc: 0.6965

Epoch 5/24
----------
train Loss: 0.8296 Acc: 0.7251
val Loss: 0.8503 Acc: 0.7195

Epoch 6/24
----------
train Loss: 0.7419 Acc: 0.7474
val Loss: 0.7857 Acc: 0.7290

Epoch 7/24
----------
train Loss: 0.6997 Acc: 0.7602
val Loss: 0.7617 Acc: 0.7375

Epoch 8/24
----------
train Loss: 0.6979 Acc: 0.7625
val Loss: 0.7514 Acc: 0.7420

Epoch 9/24
----------
train Loss: 0.6948 Acc: 0.7625
val Loss: 0.7456 Acc: 0.7415

Epoch 10/24
----------
train Loss: 0.6902 Acc: 0.7611
val Loss: 0.7411 Acc: 0.7440

Epoch 11/24
----------
train Loss: 0.6888 Acc: 0.7644
val Loss: 0.7375 Acc: 0.7440

Ep

In [33]:
torch.save(model.state_dict(),"models/"+datasetModels[nameIdx]+"_"+datasetAlgorithm[algorithmIdx]+".pt")

### JSMA

In [34]:
model.load_state_dict(trained_weights_cifar10[stateDictNames[nameIdx]])
algorithmIdx = 2

In [35]:
checkpoint = torch.load("perturbedDataset/"+datasetModels[nameIdx]+"_"+datasetAlgorithm[algorithmIdx]+datasetAppend[0])
perturbedImgCollection = checkpoint["perturbedImgCollection"]
perturbedImgCollection.requires_grad = False
perturbedLabelCollection = checkpoint["perturbedLabelCollection"]
gtLabelCollection = checkpoint["gtLabelCollection"]
gtLabelCollection = gtLabelCollection.long()

trainset = CustomDataset(perturbedImgCollection, gtLabelCollection, transform=None)
trainset, valset = torch.utils.data.random_split(trainset, [8000, 2000])

trainload = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)
valload = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=1)

dataloaders = {"train": trainload, "val": valload}
dataset_sizes = {"train": len(trainset), "val": len(valset)}

In [36]:
model.to(device)
model.train()

optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [37]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=25, device=device)
# alexnet = train_model(alexnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.1711 Acc: 0.9560
val Loss: 0.1677 Acc: 0.9545

Epoch 1/24
----------
train Loss: 0.1707 Acc: 0.9571
val Loss: 0.1702 Acc: 0.9560

Epoch 2/24
----------
train Loss: 0.1642 Acc: 0.9596
val Loss: 0.1699 Acc: 0.9565

Epoch 3/24
----------
train Loss: 0.1613 Acc: 0.9604
val Loss: 0.1686 Acc: 0.9555

Epoch 4/24
----------
train Loss: 0.1580 Acc: 0.9607
val Loss: 0.1668 Acc: 0.9560

Epoch 5/24
----------
train Loss: 0.1546 Acc: 0.9620
val Loss: 0.1649 Acc: 0.9560

Epoch 6/24
----------
train Loss: 0.1511 Acc: 0.9623
val Loss: 0.1633 Acc: 0.9550

Epoch 7/24
----------
train Loss: 0.1500 Acc: 0.9629
val Loss: 0.1624 Acc: 0.9550

Epoch 8/24
----------
train Loss: 0.1490 Acc: 0.9635
val Loss: 0.1620 Acc: 0.9550

Epoch 9/24
----------
train Loss: 0.1484 Acc: 0.9627
val Loss: 0.1618 Acc: 0.9550

Epoch 10/24
----------
train Loss: 0.1478 Acc: 0.9639
val Loss: 0.1616 Acc: 0.9550

Epoch 11/24
----------
train Loss: 0.1470 Acc: 0.9639
val Loss: 0.1614 Acc: 0.9550

Ep

In [38]:
torch.save(model.state_dict(),"models/"+datasetModels[nameIdx]+"_"+datasetAlgorithm[algorithmIdx]+".pt")

## MobileNet V3 Large Training

In [40]:
model = torchvision.models.mobilenet_v3_large(weights=torchvision.models.MobileNet_V3_Large_Weights.IMAGENET1K_V1)
num_feat = model.classifier[3].in_features
model.classifier[3] = nn.Linear(num_feat, 10)
nameIdx = 2

### DeepFool

In [42]:
model.load_state_dict(trained_weights_cifar10[stateDictNames[nameIdx]])
algorithmIdx = 0

In [43]:
checkpoint = torch.load("perturbedDataset/"+datasetModels[nameIdx]+"_"+datasetAlgorithm[algorithmIdx]+datasetAppend[0])
perturbedImgCollection = checkpoint["perturbedImgCollection"]
perturbedImgCollection.requires_grad = False
perturbedLabelCollection = checkpoint["perturbedLabelCollection"]
gtLabelCollection = checkpoint["gtLabelCollection"]
gtLabelCollection = gtLabelCollection.long()

trainset = CustomDataset(perturbedImgCollection, gtLabelCollection, transform=None)
trainset, valset = torch.utils.data.random_split(trainset, [8000, 2000])

trainload = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)
valload = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=1)

dataloaders = {"train": trainload, "val": valload}
dataset_sizes = {"train": len(trainset), "val": len(valset)}

In [44]:
model.to(device)
model.train()

optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [45]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=25, device=device)
# alexnet = train_model(alexnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.7457 Acc: 0.6204
val Loss: 0.6684 Acc: 0.8510

Epoch 1/24
----------
train Loss: 0.5840 Acc: 0.7394
val Loss: 0.5016 Acc: 0.8725

Epoch 2/24
----------
train Loss: 0.4385 Acc: 0.8354
val Loss: 0.3972 Acc: 0.8935

Epoch 3/24
----------
train Loss: 0.3498 Acc: 0.8820
val Loss: 0.3465 Acc: 0.8950

Epoch 4/24
----------
train Loss: 0.3027 Acc: 0.9059
val Loss: 0.3224 Acc: 0.8970

Epoch 5/24
----------
train Loss: 0.2751 Acc: 0.9134
val Loss: 0.3097 Acc: 0.8985

Epoch 6/24
----------
train Loss: 0.2595 Acc: 0.9175
val Loss: 0.3015 Acc: 0.9005

Epoch 7/24
----------
train Loss: 0.2504 Acc: 0.9246
val Loss: 0.2977 Acc: 0.9020

Epoch 8/24
----------
train Loss: 0.2463 Acc: 0.9258
val Loss: 0.2943 Acc: 0.9025

Epoch 9/24
----------
train Loss: 0.2497 Acc: 0.9203
val Loss: 0.2914 Acc: 0.9035

Epoch 10/24
----------
train Loss: 0.2476 Acc: 0.9215
val Loss: 0.2890 Acc: 0.9055

Epoch 11/24
----------
train Loss: 0.2464 Acc: 0.9218
val Loss: 0.2868 Acc: 0.9090

Ep

In [48]:
torch.save(model.state_dict(),"models/"+datasetModels[nameIdx]+"_"+datasetAlgorithm[algorithmIdx]+".pt")

### FGSM

In [49]:
model.load_state_dict(trained_weights_cifar10[stateDictNames[nameIdx]])
algorithmIdx = 1

In [50]:
checkpoint = torch.load("perturbedDataset/"+datasetModels[nameIdx]+"_"+datasetAlgorithm[algorithmIdx]+datasetAppend[0])
perturbedImgCollection = checkpoint["perturbedImgCollection"]
perturbedImgCollection.requires_grad = False
perturbedLabelCollection = checkpoint["perturbedLabelCollection"]
gtLabelCollection = checkpoint["gtLabelCollection"]
gtLabelCollection = gtLabelCollection.long()

trainset = CustomDataset(perturbedImgCollection, gtLabelCollection, transform=None)
trainset, valset = torch.utils.data.random_split(trainset, [8000, 2000])

trainload = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)
valload = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=1)

dataloaders = {"train": trainload, "val": valload}
dataset_sizes = {"train": len(trainset), "val": len(valset)}

In [51]:
model.to(device)
model.train()

optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [52]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=25, device=device)
# alexnet = train_model(alexnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 3.5574 Acc: 0.2680
val Loss: 2.8919 Acc: 0.3195

Epoch 1/24
----------
train Loss: 3.0524 Acc: 0.3201
val Loss: 2.4875 Acc: 0.3770

Epoch 2/24
----------
train Loss: 2.4753 Acc: 0.3914
val Loss: 2.0690 Acc: 0.4385

Epoch 3/24
----------
train Loss: 2.0193 Acc: 0.4617
val Loss: 1.7069 Acc: 0.5095

Epoch 4/24
----------
train Loss: 1.6743 Acc: 0.5182
val Loss: 1.4260 Acc: 0.5755

Epoch 5/24
----------
train Loss: 1.4118 Acc: 0.5711
val Loss: 1.2262 Acc: 0.6180

Epoch 6/24
----------
train Loss: 1.2280 Acc: 0.6119
val Loss: 1.0898 Acc: 0.6530

Epoch 7/24
----------
train Loss: 1.1309 Acc: 0.6386
val Loss: 1.0864 Acc: 0.6550

Epoch 8/24
----------
train Loss: 1.1188 Acc: 0.6382
val Loss: 1.0835 Acc: 0.6550

Epoch 9/24
----------
train Loss: 1.1077 Acc: 0.6414
val Loss: 1.0805 Acc: 0.6570

Epoch 10/24
----------
train Loss: 1.0957 Acc: 0.6466
val Loss: 1.0774 Acc: 0.6555

Epoch 11/24
----------
train Loss: 1.0889 Acc: 0.6458
val Loss: 1.0741 Acc: 0.6560

Ep

In [53]:
torch.save(model.state_dict(),"models/"+datasetModels[nameIdx]+"_"+datasetAlgorithm[algorithmIdx]+".pt")

### JSMA

In [54]:
model.load_state_dict(trained_weights_cifar10[stateDictNames[nameIdx]])
algorithmIdx = 2

In [55]:
checkpoint = torch.load("perturbedDataset/"+datasetModels[nameIdx]+"_"+datasetAlgorithm[algorithmIdx]+datasetAppend[0])
perturbedImgCollection = checkpoint["perturbedImgCollection"]
perturbedImgCollection.requires_grad = False
perturbedLabelCollection = checkpoint["perturbedLabelCollection"]
gtLabelCollection = checkpoint["gtLabelCollection"]
gtLabelCollection = gtLabelCollection.long()

trainset = CustomDataset(perturbedImgCollection, gtLabelCollection, transform=None)
trainset, valset = torch.utils.data.random_split(trainset, [8000, 2000])

trainload = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)
valload = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=1)

dataloaders = {"train": trainload, "val": valload}
dataset_sizes = {"train": len(trainset), "val": len(valset)}

In [56]:
model.to(device)
model.train()

optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [57]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=25, device=device)
# alexnet = train_model(alexnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.1918 Acc: 0.9345
val Loss: 0.1638 Acc: 0.9455

Epoch 1/24
----------
train Loss: 0.1881 Acc: 0.9360
val Loss: 0.1616 Acc: 0.9460

Epoch 2/24
----------
train Loss: 0.1863 Acc: 0.9393
val Loss: 0.1595 Acc: 0.9490

Epoch 3/24
----------
train Loss: 0.1836 Acc: 0.9406
val Loss: 0.1578 Acc: 0.9500

Epoch 4/24
----------
train Loss: 0.1774 Acc: 0.9414
val Loss: 0.1563 Acc: 0.9505

Epoch 5/24
----------
train Loss: 0.1766 Acc: 0.9403
val Loss: 0.1551 Acc: 0.9510

Epoch 6/24
----------
train Loss: 0.1715 Acc: 0.9434
val Loss: 0.1540 Acc: 0.9525

Epoch 7/24
----------
train Loss: 0.1708 Acc: 0.9425
val Loss: 0.1536 Acc: 0.9535

Epoch 8/24
----------
train Loss: 0.1699 Acc: 0.9437
val Loss: 0.1533 Acc: 0.9540

Epoch 9/24
----------
train Loss: 0.1725 Acc: 0.9407
val Loss: 0.1530 Acc: 0.9535

Epoch 10/24
----------
train Loss: 0.1686 Acc: 0.9434
val Loss: 0.1527 Acc: 0.9530

Epoch 11/24
----------
train Loss: 0.1696 Acc: 0.9433
val Loss: 0.1526 Acc: 0.9535

Ep

In [58]:
torch.save(model.state_dict(),"models/"+datasetModels[nameIdx]+"_"+datasetAlgorithm[algorithmIdx]+".pt")